In [1]:
import nest_asyncio
nest_asyncio.apply()

import flet as ft
import requests
import datetime
import sqlite3

DB_PATH = "weather_app_simple.db"
CUSTOM_REGIONS = {
    "北海道地方": ["北海道"],
    "東北地方": ["青森", "岩手", "宮城", "秋田", "山形", "福島"],
    "関東地方": ["茨城", "栃木", "群馬", "埼玉", "千葉", "東京", "神奈川", "山梨", "長野"],
    "中部地方": ["新潟", "富山", "石川", "福井", "山梨", "長野", "岐阜", "静岡", "愛知", "三重"],
    "近畿地方": ["滋賀", "京都", "大阪", "兵庫", "奈良", "和歌山", "三重"],
    "中国地方": ["鳥取", "島根", "岡山", "広島", "山口"],
    "四国地方": ["徳島", "香川", "愛媛", "高知"],
    "九州地方": ["福岡", "佐賀", "長崎", "熊本", "大分", "宮崎", "鹿児島"],
    "沖縄地方": ["沖縄"]
}

def init_db():
    conn = sqlite3.connect(DB_PATH)
    cur = conn.cursor()
    cur.execute("""
        CREATE TABLE IF NOT EXISTS forecasts (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            area_code TEXT,
            area_name TEXT,
            date TEXT,
            weather_text TEXT,
            weather_icon_type TEXT,
            created_at TEXT
        )
    """)
    conn.commit()
    conn.close()

def save_weather_data(area_code, area_name, dates, weathers):
    conn = sqlite3.connect(DB_PATH)
    cur = conn.cursor()

    cur.execute("DELETE FROM forecasts WHERE area_code = ?", (area_code,))

    now = datetime.datetime.now().isoformat()
    
    for i in range(len(dates)):
        date_str = dates[i]
        dt = datetime.datetime.fromisoformat(date_str)
        formatted_date = dt.strftime("%Y-%m-%d")
        
        weather_text = weathers[i] if i < len(weathers) else "--"
        
        if "晴" in weather_text: icon_type = "sunny"
        elif "雨" in weather_text: icon_type = "rain"
        elif "雪" in weather_text: icon_type = "snow"
        elif "曇" in weather_text: icon_type = "cloud"
        else: icon_type = "cloudy"

        cur.execute("""
            INSERT INTO forecasts (area_code, area_name, date, weather_text, weather_icon_type, created_at)
            VALUES (?, ?, ?, ?, ?, ?)
        """, (area_code, area_name, formatted_date, weather_text, icon_type, now))
    
    conn.commit()
    conn.close()

def get_forecasts_from_db(area_code):
    conn = sqlite3.connect(DB_PATH)
    conn.row_factory = sqlite3.Row
    cur = conn.cursor()
    
    cur.execute("""
        SELECT date, weather_text, weather_icon_type 
        FROM forecasts 
        WHERE area_code = ? 
        ORDER BY date ASC
    """, (area_code,))
    
    rows = cur.fetchall()
    conn.close()
    return rows

def main(page: ft.Page):
    init_db()

    page.title = "天気予報アプリ"
    page.theme_mode = ft.ThemeMode.LIGHT
    page.window_width = 1100
    page.window_height = 800
    page.padding = 20

    weather_display = ft.Column(scroll=ft.ScrollMode.AUTO, expand=True)

    def get_icon_name(icon_type):
        if icon_type == "sunny": return "wb_sunny"
        elif icon_type == "rain": return "water_drop"
        elif icon_type == "snow": return "ac_unit"
        elif icon_type == "cloud": return "cloud"
        else: return "wb_cloudy_outlined"

    def load_weather(code, name):
        weather_display.controls.clear()
        weather_display.controls.append(ft.Text(f"{name} のデータを更新中", size=20))
        page.update()

        url = f"https://www.jma.go.jp/bosai/forecast/data/forecast/{code}.json"
        
        try:
            res = requests.get(url)
            res.raise_for_status()
            data = res.json()
            
            ts_weather = data[0]['timeSeries'][0]
            dates = ts_weather['timeDefines']
            area_weather = ts_weather['areas'][0]
            weathers = area_weather['weathers']
            
            save_weather_data(code, name, dates, weathers)
            
            db_rows = get_forecasts_from_db(code)

            weather_display.controls.clear()
            weather_display.controls.append(
                ft.Text(f"{name} の天気予報", size=24, weight="bold")
            )
            
            cards_row = ft.Row(wrap=True, spacing=20, run_spacing=20)

            for row in db_rows:
                date_str = row["date"]
                weather_text = row["weather_text"]
                icon_type = row["weather_icon_type"]
                display_weather_text = weather_text.replace("　", "\n")

                card = ft.Card(
                    content=ft.Container(
                        width=200,
                        padding=20,
                        bgcolor="blue50", 
                        content=ft.Column(
                            [
                                ft.Text(date_str, size=16, weight="bold"),
                                ft.Icon(name=get_icon_name(icon_type), size=48, color="orange"),
                                ft.Text(display_weather_text, size=14, text_align="center"),
                            ],
                            alignment=ft.MainAxisAlignment.CENTER,
                            horizontal_alignment=ft.CrossAxisAlignment.CENTER,
                        ),
                    )
                )
                cards_row.controls.append(card)

            weather_display.controls.append(cards_row)
            page.update()

        except Exception as e:
            print(f"Error: {e}")
            weather_display.controls.clear()
            weather_display.controls.append(
                ft.Text(f"エラーが発生しました: {e}", color="red")
            )
            page.update()

    sidebar_content = ft.ListView(expand=True, spacing=5, padding=10)
    
    url = "http://www.jma.go.jp/bosai/common/const/area.json"
    try:
        area_res = requests.get(url)
        area_data = area_res.json()
        offices = area_data['offices']

        name_to_code = {}
        for code, info in offices.items():
            name_to_code[info['name']] = code

        for region_name, pref_list in CUSTOM_REGIONS.items():
            region_tiles = []
            for pref_search_key in pref_list:
                found_code = None
                found_name = None
                
                for api_name, code in name_to_code.items():
                    if pref_search_key in api_name:
                        found_code = code
                        found_name = api_name
                        break
                
                if pref_search_key == "北海道" and not found_code:
                    found_code = "016000"
                    found_name = "北海道(札幌)"

                if found_code:
                    tile = ft.ListTile(
                        title=ft.Text(found_name),
                        height=40,
                        on_click=lambda e, code=found_code, name=found_name: load_weather(code, name)
                    )
                    region_tiles.append(tile)
            
            if region_tiles:
                sidebar_content.controls.append(
                    ft.ExpansionTile(
                        title=ft.Text(region_name, weight="bold"),
                        controls=region_tiles,
                        initially_expanded=False,
                        text_color="black",
                    )
                )

    except Exception as e:
        sidebar_content.controls.append(ft.Text(f"地域リスト取得エラー: {e}"))

    page.add(
        ft.Row(
            [
                ft.Container(
                    content=sidebar_content,
                    width=300,
                    bgcolor="grey100",
                    border_radius=10,
                    padding=10
                ),
                ft.Container(
                    content=weather_display,
                    expand=True,
                    padding=20
                )
            ],
            expand=True,
        )
    )

if __name__ == "__main__":
    ft.app(target=main)